# Support Vector Machines (SVMs)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC, LinearSVC, SVR, LinearSVR
from sklearn.linear_model import SGDClassifier, SGDRegressor

## SVM classifiers

เราสามารถสร้าง decision boundary (hyperplane) ที่ใช้แยกข้อมูลออกเป็น class ได้หลากหลายแบบไม่จำกัด แต่ hyperplane ที่ดีที่สุดจะอยู่ห่างจากข้อมูลทุกจุดมากที่สุด (มี margin มากสุด) เราเรียก hyperplane นี้ว่า <b>maximum margin classifier</b> การวางตัวของ maximum margin hyperplane จะขึ้นกับตำแหน่งของ data point ที่อยู่ใกล้มันมากที่สุด เราเรียก data point เหล่านี้ว่า <b>support vector</b> 

<img src="../images/hyperplane.png" width="500" /><br />

Maximum margin classifier อ่อนไหวต่อ outlier มากและ overfit ดังนั้น เราจะใช้ <b>soft margin classifier</b> แทน โดยเราจะยอมให้ข้อมูลบางจุดอยู่ผิดฝั่งเมื่อเทียบกับ margin ฝั่งของ class ที่ถูกต้องของมัน โดยอยู่ในหรือนอก street ก็ได้

<img src="../images/soft_margin_classifier.png" width="300" /><br />

Data point ที่ถูก classified ผิด จะมี penalty ที่เรียกว่า <b>hinge loss</b> ($\xi$) ซึ่งจะแปรผันตามระยะห่างจาก margin ของ class ที่ถูกต้อง ส่วน hinge loss ของ data point ที่ถูก classified ได้ถูกต้อง จะเป็น 0

$$\xi_i = \max(0, 1-y_i(\boldsymbol{w} \cdot \boldsymbol{x}_i + b)))$$

- $y_i$ = label ที่ถูกต้องของ data point $i$ (เท่ากับ $\pm 1$ สำหรับ binary classification task)
- $\boldsymbol{w}$ = weight vector
- $\boldsymbol{x}_i$ = feature vector ของ data point $i$ง
- $b$ = bias

<img src="../images/hinge_loss.png" width="400" /><br />

การทำ SVM คือการหา weight vector และ bias ที่ทำให้ hinge loss ของข้อมูลทุกจุดมีค่าต่ำสุด ในขณะที่ weight vector ต้องมี norm มากที่สุดด้วย เพื่อให้ street กว้างที่สุด (margin ห่างกันมากที่สุด) เท่าที่เป็นไปได้

ความแรงของ penalty ถูกกำหนดโดย regularisation parameter $C$
- Street จะแคบลงเมื่อ $C$ มากขึ้น
- Maximum margin classifier มี $C = \pm\infty$ (street มีความกว้างเป็น 0)

<img src="../images/effect_of_regularisation_parameter.png" width="700" /><br />

เราสามารถเรียกใช้ SVM classifier ได้โดยใช้อย่างใดอย่างหนึ่งต่อไปนี้
- `sklearn.svm.SVC`
- `sklearn.svm.LinearSVC` (better optimisation)
- `sklearn.linear_model.SGDClassifier(loss="hinge")` (ใช้ SGD solver)

In [2]:
# Import data and drop duplicates
df = pd.read_csv('../data/fish_no_pikes.csv').drop_duplicates()
df

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...,...
137,Smelt,12.2,11.5,12.2,13.4,2.0904,1.3936
138,Smelt,13.4,11.7,12.4,13.5,2.4300,1.2690
139,Smelt,12.2,12.1,13.0,13.8,2.2770,1.2558
140,Smelt,19.7,13.2,14.3,15.2,2.8728,2.0672


In [3]:
# Separate features (X) and target (y) for classification task
Xc, yc = df.drop(columns='Species'), df['Species']
# Train-test split
Xc_train, Xc_val, yc_train, yc_val = train_test_split(Xc, yc, train_size=.7, random_state=42)
# Create a preprocessor to impute and scale features
preprocessor = make_pipeline(SimpleImputer(), StandardScaler())
# Encode target
encoder = LabelEncoder().fit(yc_train)
yc_train = encoder.transform(yc_train)
yc_val = encoder.transform(yc_val)

In [4]:
# Compare each SVM classifier
svm_classifiers = {
    "SVC": SVC(kernel='linear', C=10),
    "Linear SVC": LinearSVC(C=10, dual=True, max_iter=5000),
    "SGD classifier": SGDClassifier(loss='hinge', penalty='l2', alpha=1/10)
}
for name, classifier in svm_classifiers.items():
    model = make_pipeline(preprocessor, classifier).fit(Xc_train, yc_train)
    print(f"The accuracy of {name} is", accuracy_score(yc_val, model.predict(Xc_val)))

The accuracy of SVC is 0.9069767441860465
The accuracy of Linear SVC is 0.9069767441860465
The accuracy of SGD classifier is 0.7209302325581395


## SVM regressors

มีการใช้ margin ที่ตรงข้ามกับ SVM classification คือ ต้องทำให้ data point อยู่ภายใน street ให้มากที่สุด และมี $\epsilon$ เป็น hyperparameter เพิ่มอีก 1 ตัว กำหนดความกว้างของ street

<img src="../images/effect_of_epsilon.png" width="500"/><br />

เราสามารถเรียกใช้ SVM regressor ได้โดยใช้อย่างใดอย่างหนึ่งต่อไปนี้
- `sklearn.svm.SVR`
- `sklearn.svm.LinearSVR`
- `sklearn.linear_model.SGDRegressor(loss="hinge")`

In [5]:
# Separate features (X) and target (y) for regression task
Xr, yr = df.drop(columns=['Species', 'Weight']), df['Weight']
# Train-test split
Xr_train, Xr_val, yr_train, yr_val = train_test_split(Xr, yr, train_size=.7, random_state=42)

In [6]:
# Compare each SVM regressor
svm_regressors = {
    "SVR": SVR(epsilon=.1, kernel='linear', C=10),
    "Linear SVR": LinearSVR(epsilon=.1, C=10, dual=True, max_iter=5000),
    "SGD regressor": SGDRegressor(loss='squared_error', penalty='l2', alpha=1/10)
}
for name, regressor in svm_regressors.items():
    model = make_pipeline(preprocessor, regressor).fit(Xr_train, yr_train)
    print(f"The RMSE of {name} is", np.sqrt(mean_squared_error(yr_val, model.predict(Xr_val))))

The RMSE of SVR is 101.57137122595131
The RMSE of Linear SVR is 133.3215339889672
The RMSE of SGD regressor is 95.97591186369768


## More about SVMs

หากจะสร้าง SVM model (ทั้ง classifier และ regressor) <span style="color: red">ต้องทำ scaling เสมอ</span>

<img src="../images/svm_scaled_unscaled.png" width="600" /><br />

Parameter หนึ่งที่เราต้องใส่ใน SVM คือ kernel ซึ่งเป็นฟังก์ชันที่ transform ข้อมูลเพื่อสร้าง feature ใหม่ที่ทำให้เราแยกข้อมูลออกจากกันได้ง่ายขึ้น แต่การสร้างข้อมูลใหม่ทำให้ dimensionality เพิ่มขึ้น ในขณะที่เรายังมีข้อมูลเท่าเดิม kernel ที่เรานิยมใช้ เช่น
- Linear kernel เร็วที่สุด จึงใช้ได้ดีกับ high-dimensional dataset
- Polynomial kernel (ภาพด้านล่างเป็นผลจากการใช้ polynomial kernel)
- Gaussian radial basis kernel function (RBF) ใช้ได้ดีกับ non-linear dataset

<img src="../images/svm_with_kernel.png" width="600" /><br />